In [ ]:
!wget -O RecSysTermPrj.zip https://github.com/twoSilverUp/RecSysTermPrj/archive/refs/heads/main.zip

import zipfile
with zipfile.ZipFile("RecSysTermPrj.zip", 'r') as zip_ref:
    zip_ref.extractall("RecSysTermPrj")

with zipfile.ZipFile("RecSysTermPrj/RecSysTermPrj-main/dataset/GameID_1.zip", 'r') as zip_ref:
    zip_ref.extractall("dataset/")
with zipfile.ZipFile("RecSysTermPrj/RecSysTermPrj-main/dataset/GameID_2.zip", 'r') as zip_ref:
    zip_ref.extractall("dataset/")

--2025-05-06 06:13:40--  https://github.com/twoSilverUp/RecSysTermPrj/archive/refs/heads/main.zip
Resolving github.com (github.com)... 140.82.116.3
Connecting to github.com (github.com)|140.82.116.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/twoSilverUp/RecSysTermPrj/zip/refs/heads/main [following]
--2025-05-06 06:13:40--  https://codeload.github.com/twoSilverUp/RecSysTermPrj/zip/refs/heads/main
Resolving codeload.github.com (codeload.github.com)... 140.82.116.9
Connecting to codeload.github.com (codeload.github.com)|140.82.116.9|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘RecSysTermPrj.zip’

RecSysTermPrj.zip       [    <=>             ]  41.69M  40.5MB/s    in 1.0s    

2025-05-06 06:13:42 (40.5 MB/s) - ‘RecSysTermPrj.zip’ saved [43715630]



In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import ast  # 문자열 리스트 처리용

# CSV 파일 로드
df1 = pd.read_csv("dataset/df_part2.csv").drop(columns=['Unnamed: 0.1', 'Unnamed: 0', 'Image'])
df2 = pd.read_csv("dataset/df_part1.csv").drop(columns=['Unnamed: 0.1', 'Unnamed: 0', 'Image'])

In [ ]:
df = pd.concat([df2, df1], axis=0)

In [ ]:
df.head()

,Game ID,Name,Release date,Price,About the game,Supported languages,Developers,Publishers,Categories,Genres,Tags,Review,Age
0,20200,Galactic Bowling,"Oct 21, 2008",19.99,Galactic Bowling is an exaggerated and stylize...,['English'],Perpetual FX Creative,Perpetual FX Creative,"Single-player,Multi-player,Steam Achievements,...","Casual,Indie,Sports","Indie,Casual,Sports,Bowling",9 user reviews (9) - 22.2% of the 9 user revie...,0.0
1,655370,Train Bandit,"Oct 12, 2017",0.99,THE LAW!! Looks to be a showdown atop a train....,"['English', 'French', 'Italian', 'German', 'Sp...",Rusty Moyher,Wild Rooster,"Single-player,Steam Achievements,Full controll...","Action,Indie","Indie,Action,Pixel Graphics,2D,Retro,Arcade,Sc...","Positive,(24),- 95% of the 24 user reviews for...",0.0
2,1139950,Two Weeks in Painland,"Feb 3, 2020",0.00,ABOUT THE GAME Play as a hacker who has arrang...,"['English', 'Spanish - Spain']",Unusual Games,Unusual Games,"Single-player,Steam Achievements","Adventure,Indie","Indie,Adventure,Nudity,Violent,Sexual Content,...",Mostly Positive (11) - 72.7% of the 11 user re...,0.0
3,1659180,TD Worlds,"Jan 9, 2022",10.99,"TD Worlds is a dynamic, highly strategical gam...","['English', 'Russian', 'Danish']",MAKSIM VOLKAU,MAKSIM VOLKAU,"Single-player,Steam Achievements,Steam Cloud","Indie,Strategy","Tower Defense,Rogue-lite,RTS,Replay Value,Perm...",Mostly Positive (18) - 72.2% of the 18 user re...,0.0
4,1968760,Legend of Rome - The Wrath of Mars,"May 5, 2022",9.99,When the Roman people honored a simple warrior...,"['English', 'German']",magnussoft,magnussoft,"Single-player,Steam Cloud",Casual,"Casual, Rome, Match 3, Mars, 2D, Colorful, Sin...",1 user reviews (1) - 100.0% of the 1 user revi...,0.0


In [ ]:
df['Genres'] = df['Genres'].apply(lambda x: x.split(','))
df['Tags'] = df['Tags'].apply(lambda x: x.split(','))
df['Categories'] = df['Categories'].apply(lambda x: x.split(','))
df['Release date'] = pd.to_datetime(df['Release date'], format='mixed').dt.year

In [ ]:
import json

def safe_parse_list(x):
    if isinstance(x, str):
        try:
            return json.loads(x.replace("'", '"'))
        except json.JSONDecodeError:
            return []
    return x

import ast

# 문자열로 저장된 리스트를 실제 리스트로 변환
df['Supported languages'] = df['Supported languages'].apply(safe_parse_list)

In [ ]:
keywords = ["Very Positive", "Positive", "Mixed", "Mostly Negative", "Negative"]

def get_status(text):
    for keyword in keywords:
        if keyword in text:
            return keyword
    return "Need more"

df['Review'] = df['Review'].apply(get_status)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer


# TF-IDF 벡터화기기
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(df['About the game'])

In [ ]:
from sklearn.decomposition import TruncatedSVD

# TF-IDF 차원 축소. 50차원으로
svd = TruncatedSVD(n_components=50, random_state=42)
reduced_tfidf = svd.fit_transform(tfidf_matrix)

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer

# 결과 저장용 리스트
encoded_parts = [df[['Game ID', 'Name', 'Price', 'Developers', 'Publishers', 'Age', 'Release date', 'Review']]]

# 다중 리스트 컬럼들
list_columns = ['Tags', 'Categories', 'Genres', 'Supported languages']

for col in list_columns:
    mlb = MultiLabelBinarizer()
    encoded = pd.DataFrame(mlb.fit_transform(df[col]),
                           columns=[f"{col}_{cls}" for cls in mlb.classes_],
                           index=df.index)
    encoded_parts.append(encoded)

# 결과 병합
df_encoded = pd.concat(encoded_parts, axis=1)
df_encoded.head()

,Game ID,Name,Price,Developers,Publishers,Age,Release date,Review,Tags_ 1980s,Tags_ 1990's,...,Supported languages_Turkmen,Supported languages_Ukrainian,Supported languages_Urdu,Supported languages_Uyghur,Supported languages_Uzbek,Supported languages_Vietnamese,Supported languages_Welsh,Supported languages_Xhosa,Supported languages_Yoruba,Supported languages_Zulu
0,20200,Galactic Bowling,19.99,Perpetual FX Creative,Perpetual FX Creative,0.0,2008,Need more,0,0,...,0,0,0,0,0,0,0,0,0,0
1,655370,Train Bandit,0.99,Rusty Moyher,Wild Rooster,0.0,2017,Positive,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1139950,Two Weeks in Painland,0.00,Unusual Games,Unusual Games,0.0,2020,Positive,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1659180,TD Worlds,10.99,MAKSIM VOLKAU,MAKSIM VOLKAU,0.0,2022,Positive,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1968760,Legend of Rome - The Wrath of Mars,9.99,magnussoft,magnussoft,0.0,2022,Need more,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# 축소된 결과를 DataFrame으로 변환
tfidf_df = pd.DataFrame(reduced_tfidf,
                        columns=[f'tfidf_svd_{i}' for i in range(reduced_tfidf.shape[1])])

# 인덱스를 원래 df와 일치시킴
tfidf_df.index = df.index

# 원래 df에 붙이기
df_encoded = pd.concat([df_encoded, tfidf_df], axis=1)

df_encoded.rename(columns={'Game ID': 'item id'}, inplace=True)
df_encoded['Age'] = df_encoded['Age'].replace('ALL', 0).astype(float)

In [ ]:
df_encoded.head()

,item id,Name,Price,Developers,Publishers,Age,Release date,Review,Tags_ 1980s,Tags_ 1990's,...,tfidf_svd_40,tfidf_svd_41,tfidf_svd_42,tfidf_svd_43,tfidf_svd_44,tfidf_svd_45,tfidf_svd_46,tfidf_svd_47,tfidf_svd_48,tfidf_svd_49
0,20200,Galactic Bowling,19.99,Perpetual FX Creative,Perpetual FX Creative,0.0,2008,Need more,0,0,...,0.029431,0.050135,0.017827,0.036539,-0.027325,-0.004967,-0.020976,0.023484,-0.004259,0.024899
1,655370,Train Bandit,0.99,Rusty Moyher,Wild Rooster,0.0,2017,Positive,0,0,...,0.046263,0.001206,0.008600,-0.028831,-0.014004,-0.003648,-0.028129,0.000710,-0.004767,-0.030516
2,1139950,Two Weeks in Painland,0.00,Unusual Games,Unusual Games,0.0,2020,Positive,0,0,...,0.011804,-0.024508,0.015958,0.000136,-0.019678,-0.003188,0.004283,0.010729,-0.019122,0.029747
3,1659180,TD Worlds,10.99,MAKSIM VOLKAU,MAKSIM VOLKAU,0.0,2022,Positive,0,0,...,0.037654,0.030139,-0.000316,0.020422,0.022569,-0.006770,0.032415,0.025022,0.002053,0.034289
4,1968760,Legend of Rome - The Wrath of Mars,9.99,magnussoft,magnussoft,0.0,2022,Need more,0,0,...,0.002894,-0.006348,-0.021940,0.060837,-0.056502,-0.004418,-0.025923,0.016609,-0.020709,-0.049950


In [ ]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

scale = list(set(df_encoded.columns) - {'item id', 'Name', 'Developers', 'Publishers', 'Review'})

# Min-Max Scaling
scaler = MinMaxScaler()
df_encoded[scale] = scaler.fit_transform(df_encoded[scale])

In [ ]:
df_encoded

,item id,Name,Price,Developers,Publishers,Age,Release date,Review,Tags_ 1980s,Tags_ 1990's,...,tfidf_svd_40,tfidf_svd_41,tfidf_svd_42,tfidf_svd_43,tfidf_svd_44,tfidf_svd_45,tfidf_svd_46,tfidf_svd_47,tfidf_svd_48,tfidf_svd_49
0,20200,Galactic Bowling,0.01999,Perpetual FX Creative,Perpetual FX Creative,0.0,0.407407,Need more,0.0,0.0,...,0.563730,0.654096,0.523294,0.483302,0.457778,0.443676,0.327589,0.555587,0.340364,0.513726
1,655370,Train Bandit,0.00099,Rusty Moyher,Wild Rooster,0.0,0.740741,Positive,0.0,0.0,...,0.605773,0.528652,0.493697,0.330203,0.495522,0.447455,0.311577,0.505764,0.338907,0.352710
2,1139950,Two Weeks in Painland,0.00000,Unusual Games,Unusual Games,0.0,0.851852,Positive,0.0,0.0,...,0.519701,0.462726,0.517299,0.398045,0.479444,0.448773,0.384136,0.527683,0.297774,0.527812
3,1659180,TD Worlds,0.01099,MAKSIM VOLKAU,MAKSIM VOLKAU,0.0,0.925926,Positive,0.0,0.0,...,0.584270,0.602829,0.465100,0.445556,0.599147,0.438508,0.447116,0.558950,0.358451,0.541011
4,1968760,Legend of Rome - The Wrath of Mars,0.00999,magnussoft,magnussoft,0.0,0.925926,Need more,0.0,0.0,...,0.497447,0.509283,0.395738,0.540208,0.375107,0.445249,0.316515,0.540547,0.293227,0.296244
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31028,2435240,Sulphur Memories: Alchemist,0.01199,Spellweaver,Spellweaver,0.0,1.000000,Positive,0.0,0.0,...,0.493688,0.531760,0.584334,0.400704,0.530153,0.471267,0.342814,0.515412,0.366282,0.455980
31029,2214970,Get To The Gate,0.01999,Maximan,Maximan,0.0,1.000000,Need more,0.0,0.0,...,0.389197,0.657846,0.624978,0.358020,0.517893,0.521519,0.323083,0.480821,0.308068,0.490546
31030,3165130,SLIME ORDER,0.00099,かべろくん,かべろくん,0.0,1.000000,Need more,0.0,0.0,...,0.414596,0.514338,0.462607,0.482901,0.603640,0.499343,0.386704,0.677988,0.401213,0.484462
31031,2593970,Blocky Farm,0.00899,Forever Entertainment S. A.,Forever Entertainment S. A.,0.0,1.000000,Need more,0.0,0.0,...,0.538697,0.615005,0.523301,0.294460,0.602319,0.347544,0.333548,0.317377,0.776833,0.578805


In [ ]:
df_encoded.to_csv('./item_features_rechorus.csv')